Gökçe Şahin
090190306
sahingo19@itu.edu.tr

In [242]:
import pandas as pd
import wbgapi as wb
import numpy as np
import statsmodels.api as sm
import yfinance as yf

from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeRegressor

# Question 1

For this question use the World Bank Data for Turkey for the following indicators. Use [wbgapi](https://pypi.org/project/wbgapi/) for getting the data.

* [Literacy rate, adult female (SE.ADT.LITR.FE.ZS)](https://data.worldbank.org/indicator/SE.ADT.LITR.FE.ZS)
* [Labor force, female (SL.TLF.TOTL.FE.ZS)](https://data.worldbank.org/indicator/SL.TLF.TOTL.FE.ZS)
* [Poverty headcount ratio at national poverty lines (SI.POV.NAHC)](https://data.worldbank.org/indicator/SI.POV.NAHC)
* [Current health expenditure per capita (SH.XPD.CHEX.PC.CD)](https://data.worldbank.org/indicator/SH.XPD.CHEX.PC.CD)
* [GDP per capita (NY.GDP.PCAP.CD)](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD)
* [Mortality rate, under-5 (SH.DYN.MORT)](https://data.worldbank.org/indicator/SH.DYN.MORT)


Using the [statsmodels](https://www.statsmodels.org/stable/index.html) library write the best linear regression model using child mortality as the dependent variable while the rest are considered as independent variables. Pay particular attention to the fact that the order of the variables put into the model significantly impacts the performance of the model. Choose the best model by considering

* with the minimum number of variables and their interactions,
* with the optimal ordering of the independent variables and their interactions,
* $R^2$-score of the model,
* statistical significance of the model coefficients,
* ANOVA analysis of the model.


In [243]:
def extract(df, cntry, name):
    tmp = df[['time','value']][df['economy'] == cntry]
    tmp.index = tmp.time
    del tmp['time']
    tmp.columns = [[name]]
    return tmp

In [244]:
def vars_mor(cntry, dfs, vars):
    arr = [extract(dfs[i], cntry, vars[i]) for i in range(len(vars))]
    res = arr[0].join(arr[1:6])
    res.dropna(inplace=True)
    
    return res

In [245]:
var_apis = {
    'literacy': 'SE.ADT.LITR.FE.ZS',
    'labor': 'SL.TLF.TOTL.FE.ZS',
    'poverty': 'SI.POV.NAHC',
    'health': 'SH.XPD.CHEX.PC.CD',
    'GDP': 'NY.GDP.PCAP.CD',
    'mortality': 'SH.DYN.MORT'
}


In [246]:
vars = ['literacy', 'labor', 'mortality']
# independent variables + dependent variable (mortality)

In [247]:
dfs = [pd.DataFrame(list(wb.data.fetch(api))) for api in [var_apis[var] for var in vars]]

In [248]:
res = vars_mor('TUR', dfs, vars)
model = ols('mortality ~ literacy + labor', data=res).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              mortality   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                     252.1
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           1.58e-10
Time:                        23:28:17   Log-Likelihood:                -33.794
No. Observations:                  15   AIC:                             73.59
Df Residuals:                      12   BIC:                             75.71
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    147.9560      8.876     16.669      0.0

C:\Users\sahin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_stats_py.py:1772: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# Question 2

For this question use Yahoo's Finance API for the following tickers:

* Gold futures (GC=F)
* Silver futures (SI=F)
* Copper futures (HG=F)
* Platinum futures (PL=F)

1. Write the best linear regression model that explains gold futures closing prices in terms of opening prices of gold, silver, copper, and platinum futures.
2. Repeat the same for silver, copper and platinum prices.
3. Compare the models you obtained in Steps 1 and 2. Which model is better? How do you decide? Explain.

In [249]:
def tickers_openning(x, ftickers, fticker_names):
    tmp = {}
    tmp['GCF_o'] = ftickers[0]['Open']
    tmp['SIF_o'] = ftickers[1]['Open']
    tmp['HGF_o'] = ftickers[2]['Open']
    tmp['PLF_o'] = ftickers[3]['Open']
    tmp[x[:-2]+x[-1]+'_c'] = ftickers[fticker_names.index(x)]['Close']

    data = pd.DataFrame(tmp).dropna()

    return data

In [250]:
fticker_names = ['GC=F', 'SI=F', 'HG=F', 'PL=F']
ftickers = [yf.download(ticker) for ticker in fticker_names]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [251]:
[gcfc, sifc, hgfc, plfc] = [tickers_openning(x, ftickers, fticker_names) for x in fticker_names]
gcfc


,GCF_o,SIF_o,HGF_o,PLF_o,GCF_c
Date,,,,,
2000-08-30,273.899994,4.950000,0.8790,593.900024,273.899994
2000-08-31,274.799988,4.920000,0.8850,589.000000,278.299988
2000-09-01,277.000000,5.035000,0.8780,588.000000,277.000000
2000-09-05,275.799988,4.990000,0.8960,602.000000,275.799988
2000-09-06,274.200012,5.000000,0.9050,603.000000,274.200012
...,...,...,...,...,...
2022-11-01,1630.800049,19.125000,3.4945,959.799988,1645.000000
2022-11-02,1650.800049,19.780001,3.4985,960.200012,1645.699951
2022-11-03,1629.199951,19.235001,3.4455,933.400024,1627.300049


### Linear regression models that explains GC = F

In [252]:
model = ols('GCF_c ~ SIF_o + HGF_o + PLF_o', data=gcfc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  GCF_c   R-squared:                       0.878
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                 1.168e+04
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:28:21   Log-Likelihood:                -32183.
No. Observations:                4865   AIC:                         6.437e+04
Df Residuals:                    4861   BIC:                         6.440e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    431.4786      8.453     51.046      0.000     414.907     448.050
SIF_o         49.1464      0.617     79.622      0.000      47.936      50.356
HGF_o        229.0768      4.668     49.074      0.000     219.925     238.228
PLF_o         -0.6946      0.011    -61.164      0.000      -0.717      -0.672
==============================================================================
Omnibus:                      376.384   Durbin-Watson:                   0.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              874.776
Skew:                          -0.479   Prob(JB):                    1.11e-190
Kurtosis:                       4.844   Cond. No.                     3.78e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.78e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Linear regression models that explains SI = F

In [253]:
model = ols('SIF_c ~ GCF_o + HGF_o + PLF_o', data=sifc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  SIF_c   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                 1.431e+04
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:28:21   Log-Likelihood:                -11843.
No. Observations:                4865   AIC:                         2.369e+04
Df Residuals:                    4861   BIC:                         2.372e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.5914      0.129    -51.009      0.000      -6.845      -6.338
GCF_o          0.0115      0.000     79.593      0.000       0.011       0.012
HGF_o         -0.4500      0.087     -5.174      0.000      -0.621      -0.280
PLF_o          0.0109      0.000     63.846      0.000       0.011       0.011
==============================================================================
Omnibus:                     1300.011   Durbin-Watson:                   0.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6397.420
Skew:                           1.200   Prob(JB):                         0.00
Kurtosis:                       8.080   Cond. No.                     5.44e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.44e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Linear regression models that explains HG = F

In [254]:
model = ols('HGF_c ~ GCF_o + SIF_o + PLF_o', data=hgfc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  HGF_c   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     8061.
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:28:21   Log-Likelihood:                -3073.2
No. Observations:                4865   AIC:                             6154.
Df Residuals:                    4861   BIC:                             6180.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3248      0.026    -12.494      0.000      -0.376      -0.274
GCF_o          0.0015   2.96e-05     49.046      0.000       0.001       0.002
SIF_o         -0.0129      0.002     -5.466      0.000      -0.018      -0.008
PLF_o          0.0015   3.18e-05     46.056      0.000       0.001       0.002
==============================================================================
Omnibus:                      644.458   Durbin-Watson:                   0.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              953.749
Skew:                           0.973   Prob(JB):                    7.87e-208
Kurtosis:                       3.959   Cond. No.                     6.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Linear regression models that explains PL = F

In [255]:
model = ols('PLF_c ~ GCF_o + SIF_o + HGF_o', data=plfc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PLF_c   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     5763.
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:28:22   Log-Likelihood:                -31933.
No. Observations:                4865   AIC:                         6.387e+04
Df Residuals:                    4861   BIC:                         6.390e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    524.4937      6.521     80.435      0.000     511.710     537.277
GCF_o         -0.6276      0.010    -61.261      0.000      -0.648      -0.608
SIF_o         41.9496      0.657     63.839      0.000      40.661      43.238
HGF_o        207.9301      4.526     45.945      0.000     199.058     216.802
==============================================================================
Omnibus:                      247.922   Durbin-Watson:                   0.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              328.182
Skew:                           0.494   Prob(JB):                     5.45e-72
Kurtosis:                       3.802   Cond. No.                     3.49e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.49e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

***
Linear regression model that explains SI=F closing prices is better than the others because its R^2 value is larger.
***

# Question 3

1. Write a function that takes a ticker symbol and returns a pandas dataframe that for each day puts a 1 when the closing price is higher than the opening price, a 0 when the closing price is lower than the opening price.
2. Write the best logistic regression that predicts the time series you obtain from Step 1 for gold futures against the opening prices of gold, silver, copper, and platinum prices.
3. Repeat the same for silver, copper, and platinum prices.
4. Compare the models you obtained from Steps 2 and 3. Decide which is the best model, and explain your reasoning.
5. Does any of the models provide a good fit? Explain.

In [256]:
def openning_closing(ticker):
    new_header = ticker[:-2]+ticker[-1]+'_co'

    tmp = yf.download(ticker)
    tmp[new_header] = 0

    tmp[new_header][tmp['Close'] > tmp['Open']] = 1

    tmp_co = pd.DataFrame(tmp[new_header])

    return tmp_co

In [257]:
def ticker_co_df(ticker, tickers_df):
    df = pd.concat([tickers_df, openning_closing(ticker)], axis = 1)
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    del df['Date']
    return df


In [258]:
def bootstrap(X, y, model):
    res = []
    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)
        model.fit(X_train,y_train)
        res.append(model.score(X_test,y_test))
    tmp = sorted(res)[3:97]
    return (min(tmp),max(tmp))

In [259]:
def print_model(df):
    X = df.iloc[:, :4]
    y = df.iloc[:, 5]

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

    model = LogisticRegression(max_iter=4000)
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))
    
    y_predict = model.predict(X_test)
    print(confusion_matrix(y_test, y_predict))

    print(bootstrap(X, y, model))


In [260]:
fticker_names = ['GC=F', 'SI=F', 'HG=F', 'PL=F']
ftickers = [yf.download(ticker) for ticker in fticker_names]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [261]:
# I used tickers_openning() function from question-2
gcf_df = tickers_openning('GC=F', ftickers, fticker_names)
sif_df = tickers_openning('SI=F', ftickers, fticker_names)
hgf_df = tickers_openning('HG=F', ftickers, fticker_names)
plf_df = tickers_openning('PL=F', ftickers, fticker_names)

In [262]:
gcf_co_df = ticker_co_df('GC=F', gcf_df)
print_model(gcf_co_df)

[*********************100%***********************]  1 of 1 completed
0.5784716516023007
[[614  75]
 [438  90]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_2116\4014248705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.5546425636811833, 0.5916187345932621)


In [263]:
sif_co_df = ticker_co_df('SI=F', sif_df)
print_model(sif_co_df)

[*********************100%***********************]  1 of 1 completed
0.6409202958093673
[[777  13]
 [424   3]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_2116\4014248705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.6409202958093673, 0.6803615447822514)


In [264]:
hgf_co_df = ticker_co_df('HG=F', hgf_df)
print_model(hgf_co_df)

[*********************100%***********************]  1 of 1 completed
0.485620377978636
[[571  18]
 [608  20]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_2116\4014248705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.48315529991783074, 0.5291700903861956)


In [265]:
plf_co_df = ticker_co_df('PL=F', plf_df)
print_model(plf_co_df)

[*********************100%***********************]  1 of 1 completed
0.723911257189811
[[808  79]
 [257  73]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_2116\4014248705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.7091207888249794, 0.7460969597370584)


* Model for the PL=F seems to be the best according to their scores.
* Values that are estimated to be 0 but actually 1 are too much in all four models. As a result, models that predicted 0 more get worse results. For this reason, we cannot say that any of the models provide a good fit.

# Question 4

For this question use the following [data](https://archive.ics.uci.edu/ml/datasets/credit+approval):


In [266]:
credit = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', header=None)

fn = {'+': 1, '-': 0}

X = credit.replace('?',0).iloc[:,[1,2,7,10,14]]
y = credit.iloc[:,15].map(lambda x: fn.get(x,0))

1. Split the data into training and test set.
2. Write different logistic regression models predicting y against X.
3. Construct [confusion matrices](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) on the test data set for these different models.
4. Analyze these models. Explain which model is the best model you have found.
5. Repeat Steps 1-4 several times. Does your best model stay as the best model? What should be the correct protocol to decide on the best model explaining the data?

In [267]:
X = X.astype(float)

In [268]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

***

In [269]:
model1 = LogisticRegression()
model1.fit(X_train, y_train)
model1.score(X_test, y_test)

0.7283236994219653

In [270]:
y_predict = model1.predict(X_test)
confusion_matrix(y_test, y_predict)

array([[86, 11],
       [36, 40]], dtype=int64)

***

In [271]:
df = pd.concat([X_train, y_train], axis=1)
df.columns = ['a','b','c','d','e','p']

In [272]:
model2 = smf.logit('p ~ a + b + c + d + e', data=df).fit()
model2.summary()

Optimization terminated successfully.
         Current function value: 0.476230
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      p   No. Observations:                  517
Model:                          Logit   Df Residuals:                      511
Method:                           MLE   Df Model:                            5
Date:                Mon, 07 Nov 2022   Pseudo R-squ.:                  0.3073
Time:                        23:28:43   Log-Likelihood:                -246.21
converged:                       True   LL-Null:                       -355.43
Covariance Type:            nonrobust   LLR p-value:                 3.223e-45
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.9521      0.329     -5.931      0.000      -2.597      -1.307
a              0.0100      0.010      1.039      0.299      -0.009       0.029
b              0.0282      0.024      1.177      0.239      -0.019       0.075
c              0.1985      0.045      4.399      0.000       0.110       0.287
d              0.3934      0.055      7.211      0.000       0.287       0.500
e              0.0005      0.000      3.735      0.000       0.000       0.001
==============================================================================
"""

***

In [273]:
model3 = sm.Logit(y_train, X_train).fit()
model3.summary()

Optimization terminated successfully.
         Current function value: 0.513398
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     15   No. Observations:                  517
Model:                          Logit   Df Residuals:                      512
Method:                           MLE   Df Model:                            4
Date:                Mon, 07 Nov 2022   Pseudo R-squ.:                  0.2532
Time:                        23:28:43   Log-Likelihood:                -265.43
converged:                       True   LL-Null:                       -355.43
Covariance Type:            nonrobust   LLR p-value:                 7.463e-38
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
1             -0.0411      0.006     -7.376      0.000      -0.052      -0.030
2             -0.0108      0.023     -0.469      0.639      -0.056       0.034
7              0.2091      0.043      4.809      0.000       0.124       0.294
10             0.3625      0.051      7.154      0.000       0.263       0.462
14             0.0004      0.000      3.377      0.001       0.000       0.001
==============================================================================
"""

In [274]:
y_predict3 = model3.predict(X_test)
y_predict3 = np.where(y_predict3 > 0.5, 1, 0)
confusion_matrix(y_test, y_predict3)

array([[78, 19],
       [31, 45]], dtype=int64)

***

In [275]:
model4 = DecisionTreeRegressor()
model4.fit(X_train, y_train)
bootstrap(X, y, model4)

(-0.5227994227994228, 0.028089887640449507)

In [276]:
y_predict4 = model4.predict(X_test)
confusion_matrix(y_test, y_predict4)

array([[92,  5],
       [ 7, 69]], dtype=int64)

***

In [277]:
model5 = LinearRegression()
model5.fit(X_train, y_train)
bootstrap(X, y, model5)

(-0.020744153891729677, 0.28762742519796947)

In [278]:
y_predict5 = model5.predict(X_test)
y_predict5 = np.where(y_predict5 > 0.5, 1, 0)
confusion_matrix(y_test, y_predict5)

array([[88,  9],
       [41, 35]], dtype=int64)

***

Binary logistic regression model is the best model because y has 2 values (1 and 0). DecisionTreeRegressor() predicts y better than the others.

What should be the correct protocol to decide on the best model explaining the data?
* $R^2$-score of the model should be large,
* p-values of the model coefficients should be small
* the diagonal of the confusion matrix should have the largest numbers